# Data Cleaning and Preparation

## Objective
This notebook focuses on cleaning the book dataset by handling missing values,
removing duplicates, correcting data types, and improving overall data quality.
The cleaned dataset will be used for further analysis and machine learning.


In [3]:
import pandas as pd
import numpy as np

In [4]:
data_path = "../data/processed/books_initial.csv"
df = pd.read_csv(data_path)

df.head()

,Unnamed: 0,bookId,title,author,series,description,genres,awards,characters,places,...,publish_date,num_pages,num_ratings,num_reviews,avg_rating,rated_1,rated_2,rated_3,rated_4,rated_5
0,0,1,Harry Potter and the Half-Blood Prince,J.K. Rowling,Harry Potter #6,The war against Voldemort is not going well; e...,"Art,Biography,Business,Children's,Christian,Cl...",Locus Award Nominee for Best Young Adult Novel...,"Draco Malfoy,Ron Weasley,Petunia Dursley,Verno...","Hogwarts School of Witchcraft and Wizardry,Eng...",...,September 16th 2006,652.0,2553909.0,41470.0,4.57,13147.0,29020.0,174312.0,608825.0,1728605.0
1,1,2,Harry Potter and the Order of the Phoenix,"J.K. Rowling,Mary GrandPré",Harry Potter #5,There is a door at the end of a silent corrido...,"Art,Biography,Business,Children's,Christian,Cl...",Bram Stoker Award for Works for Young Readers ...,"Sirius Black,Draco Malfoy,Ron Weasley,Petunia ...","Hogwarts School of Witchcraft and Wizardry,Lon...",...,September 2004,870.0,2631427.0,44793.0,4.50,16236.0,41738.0,231438.0,665628.0,1676387.0
2,2,3,Harry Potter and the Sorcerer's Stone,"J.K. Rowling,Mary GrandPré",Harry Potter #1,Harry Potter's life is miserable. His parents ...,"Art,Biography,Business,Children's,Christian,Cl...",Mythopoeic Fantasy Award for Children's Litera...,"Draco Malfoy,Ron Weasley,Petunia Dursley,Verno...","London, England,Hogwarts School of Witchcraft ...",...,November 1st 2003,309.0,7434783.0,117823.0,4.48,126344.0,147184.0,627803.0,1694206.0,4839246.0
3,3,4,Harry Potter and the Chamber of Secrets,J.K. Rowling,Harry Potter #2,The Dursleys were so mean and hideous that sum...,"Art,Biography,Business,Children's,Christian,Cl...",Mythopoeic Fantasy Award for Children's Litera...,"Draco Malfoy,Ron Weasley,Petunia Dursley,Verno...","Hogwarts School of Witchcraft and Wizardry,Hog...",...,November 1st 2003,352.0,2878196.0,55839.0,4.43,15623.0,55060.0,315917.0,787181.0,1704415.0
4,4,5,Harry Potter and the Prisoner of Azkaban,"J.K. Rowling,Mary GrandPré",Harry Potter #3,"For twelve long years, the dread fortress of A...","Art,Biography,Business,Children's,Christian,Cl...",Bram Stoker Award for Best Work for Young Read...,"Sirius Black,Draco Malfoy,Ron Weasley,Petunia ...","Hogwarts School of Witchcraft and Wizardry,Lon...",...,May 1st 2004,435.0,2972519.0,58483.0,4.57,14124.0,29105.0,214151.0,701765.0,2013374.0


In [5]:
print("Shape before cleaning:", df.shape)

Shape before cleaning: (20068, 24)


In [6]:
missing_before = df.isnull().sum()
missing_before

Unnamed: 0                0
bookId                    0
title                   559
author                  561
series                15789
description            2687
genres                  256
awards                16939
characters            15289
places                16028
isbn                   2537
isbn13                  858
language               2779
first_publish_date     5477
publish_date           1046
num_pages              1640
num_ratings             559
num_reviews             559
avg_rating              559
rated_1                3430
rated_2                3430
rated_3                3430
rated_4                3430
rated_5                3430
dtype: int64

In [7]:
duplicate_count = df.duplicated().sum()
print("Duplicate rows:", duplicate_count)

Duplicate rows: 0


In [8]:
# Fill missing language with 'Unknown'
df['language'] = df['language'].fillna('Unknown')

# Fill missing numeric columns with median
numeric_cols = ['avg_rating', 'num_ratings', 'num_reviews', 'num_pages']
for col in numeric_cols:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median())

In [9]:
df = df.drop_duplicates()

In [10]:
# Convert publication dates to datetime
date_cols = ['first_publish_date', 'publish_date']

for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

In [11]:
missing_after = df.isnull().sum()
missing_after

Unnamed: 0                0
bookId                    0
title                   559
author                  561
series                15789
description            2687
genres                  256
awards                16939
characters            15289
places                16028
isbn                   2537
isbn13                  858
language                  0
first_publish_date    17327
publish_date          11494
num_pages                 0
num_ratings               0
num_reviews               0
avg_rating                0
rated_1                3430
rated_2                3430
rated_3                3430
rated_4                3430
rated_5                3430
dtype: int64

In [12]:
print("Shape after cleaning:", df.shape)

Shape after cleaning: (20068, 24)


In [13]:
cleaning_summary = pd.DataFrame({
    "Metric": ["Total Rows", "Missing Values", "Duplicate Rows"],
    "Before Cleaning": [
        missing_before.shape[0],
        missing_before.sum(),
        duplicate_count
    ],
    "After Cleaning": [
        df.shape[0],
        missing_after.sum(),
        0
    ]
})

cleaning_summary

,Metric,Before Cleaning,After Cleaning
0,Total Rows,24,20068
1,Missing Values,101272,117474
2,Duplicate Rows,0,0


In [14]:
df.to_csv("../data/processed/books_clean.csv", index=False)